# 检查点
本文档介绍了如何保存和恢复通过 Estimator 构建的 TensorFlow 模型。

In [1]:
import tensorflow as tf
import pandas as pd
import os

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL =  "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES =  ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
def load_data(label_name = 'Species'):
    train_path = tf.keras.utils.get_file(fname=os.path.basename(TRAIN_URL),
                                        origin = TRAIN_URL)
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    
    train_features,train_label = train,train.pop(label_name)
    
    test_path = tf.keras.utils.get_file(fname=os.path.basename(TEST_URL),
                                       origin=TEST_URL)
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    
    test_features,test_label = test,test.pop(label_name)
    
    return (train_features,train_label),(test_features,test_label)

In [3]:
(train_x,train_y),(test_x,test_y) = load_data()
print(train_x[:5])

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


In [4]:
my_feature_column = []
for key in train_x.keys():
    my_feature_column.append(tf.feature_column.numeric_column(key))

# 检查点频率

In [14]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs=20*60,
    keep_checkpoint_max=10
)
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_column,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir ='models/iris',
    config = my_checkpointing_config)
print(classifier.model_dir)

INFO:tensorflow:Using config: {'_model_dir': 'models/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1200, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D1AB63F748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
models/iris


In [6]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [7]:
classifier.train(
    input_fn=lambda:train_input_fn(train_x,train_y,batch_size=1000),
    steps = 50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/iris\model.ckpt.
INFO:tensorflow:loss = 1924.5448, step = 1
INFO:tensorflow:Saving checkpoints for 50 into models/iris\model.ckpt.
INFO:tensorflow:Loss for final step: 297.52673.


In [8]:
def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (dict(features), labels)

    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [9]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, 1000))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-08-07:04:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris\model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-08-07:04:56
INFO:tensorflow:Saving dict for global step 50: accuracy = 0.96666664, average_loss = 0.31464368, global_step = 50, loss = 9.43931
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 50: models/iris\model.ckpt-50

Test set accuracy: 0.967



In [10]:
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}
expected = SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [11]:
predictions=classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x,None,batch_size=1000))

In [12]:
for pred_dict,expc in zip(predictions,expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    print(('\nPrediction is "{}" ({:.1f}%), expected "{}"').format(SPECIES[class_id],100*probability,expc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris\model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (90.2%), expected "Setosa"

Prediction is "Versicolor" (67.6%), expected "Versicolor"

Prediction is "Virginica" (62.5%), expected "Virginica"
